<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/3_5_PySpark_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Warmup
Import the numpy, pandas, matplotlib and pyspark packages in the cell below.

#Exercise
We are going to be using data on Student Performance during exams. Obtain the data from the following url and set up a Schema for it which you will load into pySpark. https://www.kaggle.com/spscientist/students-performance-in-exams. Set up the PySpark object and create a DataFrame on PySpark for the dataset.

Get an average score for each student using their math, reading and writing scores. 

Get a Count of the Number of Boys and girls in the class

Apply the rank function over the overall average score that you had calculated previously.

Do the same exercises above but using a Pandas DataFrame instead.

#Python Practice Question
Concordia University has the following grading policy:

Every student receives a grade in the inclusive range from 0 to 100 . Any less than 40 is a failing grade.

Sam is a professor at the university and likes to round each student's grade according to these rules: If the difference between the grade and the next multiple of 5 is less than 3, round up to the next multiple of 5. If the value of the grade is less than 38, no rounding occurs as the result will still be a failing grade. For example, grade = 84 will be rounded to 85 but grade = 29 will not be rounded because the rounding would result in a number that is less than 40.

Given the initial value of grade for each of Sam's students, write code to automate the rounding process.
Examples



```
round to  (85 - 84 is less than 3)
 do not round (result is less than 40)
 do not round (60 - 57 is 3 or higher)
```




```
Sample Input
0
4
73
67
38
33

Sample Output 
0
4
75
67
40
33
```





In [6]:
def Students(grades):

    for x,y in enumerate(grades):
        if(y>=38) and (y%5)>=3:
            grades[x]=y+5-(y%5)
    return (grades)

grades = [0, 4, 73, 67, 38, 33]
Students(grades)

[0, 4, 75, 67, 40, 33]